In [1]:
import ipywidgets
import IPython
import unray
import dolfin
import numpy as np

from dolfin import *
from unray import Data, DataDisplay, render
from IPython.display import display

print(ipywidgets.__version__)
print(IPython.__version__)
print(unray.__version__)

7.0.0a4
6.1.0
0.1.0.dev


In [2]:
def create_wave_mesh(N, M, xfreq, yfreq, amp):
    mesh = UnitCubeMesh(N, N, M)
    print(mesh.num_cells(), mesh.num_vertices())
    coordinates = mesh.coordinates()
    coordinates[:,2] += np.sin(3.14159*xfreq*coordinates[:,0])
    coordinates[:,2] += np.sin(3.14159*yfreq*coordinates[:,1])
    coordinates[:,2] *= amp
    return mesh

In [3]:
mesh = create_wave_mesh(32, 4, 4.0, 3.0, 0.125)
#mesh = Mesh("data/heart_mesh.xml.gz")
#mesh = Mesh("data/dog_mesh_97k.xml.gz")
#mesh = UnitCubeMesh(1, 1, 1)
#mesh = UnitTetrahedronMesh()

24576 5445


In [4]:
r = 0
for i in range(r):
    mesh = refine(mesh)
print(mesh.num_cells(), mesh.num_vertices())
V = FunctionSpace(mesh, "P", 1)
f0 = interpolate(Expression("1.0", degree=1), V)
f = interpolate(Expression("x[0]", degree=1), V)
g = interpolate(Expression("x[1]", degree=1), V)

24576 5445


In [5]:
cells = mesh.cells()
coordinates = mesh.coordinates()
density = f.compute_vertex_values()
emission = g.compute_vertex_values()
density_lut = None
emission_lut = None

#density_lut = np.asarray([1.0, 1.0, 0.0, 0.0, 1.0, 1.0], dtype="float32")
#density_lut = np.asarray([0.0, 0.0, 1., 1., 0.0, 0.0], dtype="float32")
#density_lut = np.asarray([1, 1, 1, 1], dtype="float32")
density_lut = np.asarray([0.2, .4, .6, .8], dtype="float32")

emission_lut = np.asarray([[1,1,0], [1,.3,.6], [1,.6,.3], [1,1,0]], dtype="float32")
#emission_lut = np.asarray([[1,1,0], [0,1,0], [0,0,1], [1,1,1]], dtype="float32")
#emission_lut=np.asarray([[1,1,.9], [0,.5,0], [.5,.5,.5], [0,1,1]], dtype="float32")


In [12]:
mesh.num_cells()

24576

In [48]:
cell_indicators = np.zeros(mesh.num_cells(), dtype="int32")
cell_indicators[:] = 2
cell_indicators[:3000] = -2
cell_indicators[-3000:] = -2
#cell_indicators[::2] = 1
#cell_indicators[:mesh.num_cells()//2] = -2
cell_indicators

array([-2, -2, -2, ..., -2, -2, -2], dtype=int32)

In [49]:
method = "cells"
fig = render(coordinates, cells,
             density=density, emission=emission,
             density_lut=density_lut,
             emission_lut=emission_lut,
             cell_indicators=cell_indicators,
             width=400, height=400,
             method=method)
fig.animate = False
#fig.animate = True
fig

A Jupyter Widget

In [ ]:
fig.animate = False


In [ ]:
figs = []
#for method in ["min", "min2", "xray", "xray2"]:
#for method in ["isosurface2", "max2", "xray2", "min2"]:
#for method in ["isosurface", "max", "xray", "min"]:
#for method in ["surface", "surface_depth", "isosurface", "xray"]:
#for method in ["surface", "surface_depth", "isosurface2", "xray2"]:
for method in ["surface_depth", "surface", "max", "isosurface"]:
    fig = render(coordinates, cells,
                 density=density, emission=emission,
                 density_lut=density_lut,
                 emission_lut=emission_lut,
                 width=400, height=400,
                 method=method)
    fig.animate = True
    figs.append(fig)
box = ipywidgets.HBox([ipywidgets.VBox(figs[:2]), ipywidgets.VBox(figs[2:])])
box

In [ ]:
for fig in figs:
    fig.animate = False

In [ ]:
for fig in figs:
    fig.animate = True

In [ ]:
[(coordinates[:,i].min(), coordinates[:,i].max()) for i in (0,1,2)]

In [ ]:
import itertools
comb = list(itertools.permutations(range(4), 4))
ncomb = len(comb)
k = 0
print("computed permutations:", comb)
print("ncomb:", ncomb)

coordinates = np.asarray([[0,0,0],  [1,0,0],  [0,1,0],  [.5,.5,1]], dtype="float32")
density = np.asarray([0.0, .33, .66, 1.0], dtype="float32")
emission = np.asarray([0.0, .33, .66, 1.0], dtype="float32")
density_lut = None
emission_lut = np.asarray([[1,0,1], [1,.3,.6], [1,.6,.3], [1,1,0]], dtype="float32")

figs = {}
m = 0
#m = 8
for k in range(m, m+16):
    cells = np.asarray([[0, 1, 2, 3]], dtype="int32")

    #perm = [0,1,2,3]
    perm = list(comb[k])
    print("using perm ", k, ": ", perm)
    cells[0][:] = cells[0][perm]

    fig = render(coordinates, cells,
                 density=density, emission=emission,
                 density_lut=density_lut,
                 emission_lut=emission_lut,
                 width=64, height=64)
    figs[k] = fig
    fig.animate = False
    #display(fig)

box = ipywidgets.VBox([ipywidgets.HBox([figs[m+k] for k in range(i*4,i*4+4)]) for i in range(4)])
display(box)

In [ ]:
fig.animate = True

In [ ]:
fig.animate = False

In [ ]:
coordinates = np.asarray([[0,0,0],  [1,0,0],  [0,1,0],  [.5,.5,1], [.5, .5, -1]], dtype="float32")
density = np.asarray([0.0, .33, .66, 1.0, 1.0], dtype="float32")
emission = np.asarray([0.0, .33, .66, 1.0, 1.0], dtype="float32")
density_lut = None
emission_lut = np.asarray([[1,0,1], [1,.3,.6], [1,.6,.3], [1,1,0]], dtype="float32")

cells = np.asarray([[0, 1, 2, 3], [0, 1, 2, 4]], dtype="int32")

fig = render(coordinates, cells,
             density=density, emission=emission,
             density_lut=density_lut,
             emission_lut=emission_lut,
             width=512, height=512)
fig.animate = False
display(fig)

In [ ]:
fig.animate = True

In [ ]:
fig.animate = False

In [ ]:
fig.update_data(emission_lut=np.asarray([[1,0,0], [0,1,0], [0,0,1], [1,1,1]], dtype="float32"))

In [ ]:
mesh

In [ ]:
fig.update_data(emission_lut=np.asarray([[1,1,.9], [0,0,0], [.5,.5,.5], [0,1,1]], dtype="float32"))

In [ ]:
cells = Data(name="cells", array=cells_array)
coordinates = Data(name="coordinates", array=coordinates_array)
density = Data(name="density", array=density_array)

In [ ]:
density_display = DataDisplay(data=density)
#density_display

In [ ]:
density_array[:] *= 0.8
density.send_state("array")